In [8]:
import tkinter 
from tkinter import ttk 
import sqlite3
import pandas as pd

choosed= 0

def closeFrame(self):
    self.pack_forget()
    
def opendb():
    global con, cur
    con = sqlite3.connect('mainbase.db')
    cur = con.cursor()

    return con,cur

def openFrame(self,sF = 0):
    '''Abre uma nova janela

        @param self: Janela que sera aberta

        @param sF se é um sub Frame (default 0)

    '''    
    
    global nowFrame
    if sF == 0:
        closeFrame(nowFrame)
    self.pack(fill='both', expand= True)
    
    nowFrame = self
    return(nowFrame)

def seachStd(self, typ, datab = 0, filt= 0):
    ''' pesquisa por um dado dentro do banco.
        @param typ: tipo de pesquisa : 0 basica, com saida em toast; 1 complexa, com filtos, saidas no menu de exibição.
        @param self: Entry qual sera buscado. 
        @param datab: base qual sera pesquisada; EX : estudantes, professores, disciplina
        @param filt: filtro selecionado
    '''
    
    if typ == 0:
        opendb()   
        stdget = self.get()
        cur.execute("SELECT nome FROM estudantes WHERE nome LIKE ?", ('%' + stdget + ' %',))
        founded = cur.fetchone()
        con.close()
        if founded:
            tstMensage(0, "O Aluno foi ENCONTRADO !")
            print(founded)
        else:
            tstMensage(0,'        O Aluno não foi encontrado; \n tente ver se foi escrito corretamente.')
    
                    
    elif typ == 1:
    
        #closeFrame(rsFrame)
        opendb()
        stdget = self.get()
        stdget = ("'%"+stdget+"%'")
        datab= datab.get()
        filt = filt.get()
        query = """
            SELECT * 
            FROM %s 
            WHERE %s LIKE %s
            """ %(datab, filt, stdget)
        df = pd.read_sql_query(query, con)
        con.close()
        
        rsFrame = tkinter.Frame(shwFrame)
        rsFrame.grid(row=4, column=1)

        rowSet= df.to_numpy().tolist()
        rsTree = ttk.Treeview(rsFrame, show='headings', columns=l1,height=3)
        rsTree.grid()

        for i in l1:
            rsTree.column(i, width=90, anchor='c')
            rsTree.heading(i, text= str(i))
        for dt in rowSet:
            v = [r for r in dt]
            rsTree.insert("", 'end', iid= v[0], values= v)      
    else:
        tstMensage(2,'tipo de pesquisa incorreto')
        
def prtList():
    
    opendb()
    dbcur = con.cursor()
    dbcur.execute("SELECT * FROM estudantes;")
    print('=========================================================')
    for line in dbcur.fetchall():
        print(line)
    print('=========================================================')
    con.close()
    
def shwList(typ):
    ''' exibe em formato de lista a tabela
        @parm typ: base a ser buscada; 0 alunos, 1 professores, 2 diciplinas;  
    '''

    
    global shwSFtlist,shwSFrame,choosed, l1
    
    openFrame(shwFrame)
    

    if typ == 1:
        opendb()
        df = pd.read_sql_query("SELECT * FROM professores ORDER BY id ASC",con)
        con.close()
        l1= list(df)
    elif typ == 2:
        opendb()
        df = pd.read_sql_query("SELECT * FROM diciplinas ORDER BY id ASC",con)
        con.close()
        l1= list(df)
    else:
        opendb()
        df = pd.read_sql_query("SELECT * FROM estudantes ORDER BY id ASC",con)
        con.close()
        l1= list(df)

    rowSet= df.to_numpy().tolist()

    

    shwSFtlist = ttk.Treeview(shwSFrame, show='headings', columns=l1,height=10)
    shwSFtlist.grid(row= 2, column=1)

    for i in l1:
        shwSFtlist.column(i, width=90, anchor='c')
        shwSFtlist.heading(i, text= str(i))
    for dt in rowSet:
        v = [r for r in dt]
        shwSFtlist.insert("", 'end', iid= v[0], values= v)

def upd():
    for i in shwSFtlist.get_children():
        shwSFtlist.delete(i)

def tstMensage(typ, msg):
    ''' exibe um pop up na tela.

        @param typ: qual será o tipo de interface; 0 mensagem, 1 confirmação, 2 erro.
        @param msg: mensagem que sera mostrada 
    '''

    global toast
    if typ == 0:
        toast = tkinter.Tk()
        toast.title('MENSAGEM!')
        toast.geometry('200x70')
        msg = tkinter.Label(toast, text= msg)
        msg.pack(side='top')
        tstButton = tkinter.Button(toast, text='OK', command= lambda:toast.destroy())
        tstButton.pack(side='bottom', pady=10)
        toast.mainloop()
    elif typ == 1:
        global conf
        conf = False
        toast = tkinter.Tk()
        toast.title('CONFIRMAÇÃO!')
        toast.geometry('270x80')
        msg = tkinter.Label(toast, text= msg)
        msg.pack(side='top')
        msg1 = tkinter.Label(toast, text='deseja confirmar ?')
        msg1.pack(side='top')
        tstButton = tkinter.Button(toast, text='cancelar', padx=5, command= lambda:[tstConf(False), toast.destroy()])
        tstButton.pack(side='left', pady=10)
        tstButton1 = tkinter.Button(toast, text='ok', padx=20, command=lambda:[tstConf(True), toast.destroy()])
        tstButton1.pack(side='right', pady=10)

       # toast.mainloop()

        return(conf, toast)
    elif typ == 2:
        toast = tkinter.Tk()
        toast.title('OPS!')
        toast.geometry('200x70')
        msg0= tkinter.Label(toast, text='Algo deu errado. ')
        msg0.pack(side='top')
        msg = tkinter.Label(toast, text= 'detalhes :' + msg)
        msg.pack(side='top')
        tstButton = tkinter.Button(toast, text='OK', command= lambda:toast.destroy())
        tstButton.pack(side='bottom', pady=10)

        toast.mainloop()

def tstConf(val):
    ''' altera variavel conf para True ou False
        @param val: True ou False
    '''
    global conf

    if val:
        conf= True
    else:
        conf= False

    return conf    

def shwTyp():
    ''' altera variavel choosed 
    '''
    global choosed, sFRSpinbox
    val = sFRSpinbox.get()

    if val == 'professores':
        choosed= 1
        '''elif val == 'diciplinas':
        choosed= 2'''
    else:
        choosed = 0

    

    return choosed  

def ftMat():


    return

def newId(database):
    global shwSFtlist,nid
    opendb()


    nid1 = len(cur.execute('SELECT * from %s' %(database)).fetchall())
    nid1 = nid1+1
    print('linhas a serem validadas : %s ' %(nid1))
    for i in range(nid1):
        print('atual i : %s' %(i))
        cur.execute("SELECT id FROM %s WHERE id LIKE %s" %(database,i))
        founded = cur.fetchone()
        if founded:
            print('row %s esta oculpada' %(i))
        else:
            print('row limpa %s' %(i))
            nid = i
            break     
    con.close()
    return nid

def newStd(typ):
        ''' cria um novo castro no banco de dados

            @param typ: tipo de cadastro; 0 alunos, 1 professor, 2 disciplina
        
        '''       
        stdname = stdBxname.get()
        stdmtr = stdBxmtr.get()
        stdid = stdBxid.get()
        
        if typ == 0:
            newId('estudantes')
            opendb()
            cur.execute("INSERT INTO estudantes (id, nome, matricula, senha) VALUES (?, ?, ?, ?)", (nid, stdname + " ", stdmtr, stdid))
            cur.execute("SELECT nome FROM estudantes WHERE nome LIKE ?", ('%' + stdname + '%',))
            founded = cur.fetchone()
            con.commit()
            con.close()
            if founded:
                tstMensage(0, 'O Aluno foi cadastrado com sucesso')
            else:
                tstMensage(2, 'Nao foi possivel criar o cadastro do Aluno. \n Ouve algum erro.')
        elif typ == 1:
            newId('professores')
            opendb()
            cur.execute("INSERT INTO professores (id, nome, matricula, senha) VALUES (?, ?, ?, ?)", (nid, stdname, stdmtr, stdid))
            cur.execute("SELECT nome FROM professores WHERE nome LIKE ?", ('%' + stdname + '%',))
            founded = cur.fetchone()
            con.commit()
            con.close()
            if founded:
                tstMensage(0, 'O Professor foi cadastrado com sucesso')
            else:
                tstMensage(2, 'Nao foi possivel criar o cadastro do Professor. \n Ouve algum erro.')

def entryFrame(self,typ):
    ''' abre o Frame de cadastro 
    
        @param self: O frame a qual pertence.

        @param typ: o tipo de cadastro, sendo 0 estudantes , 1 professor

    ''' 
    global stdBxname, stdBxmtr, stdBxid
    stdFnew = tkinter.Frame(self, width= 725, height= 400, bg= 'white')
    stdFnew.pack(fill='both', expand=True)
    stdLname = tkinter.Label(stdFnew, text="Nome :" , background= "white", font= "Inder 12")
    stdLname.grid(row= 0, column= 0, pady= 2, stick='w')
    stdBxname = tkinter.Entry(stdFnew, width=20, background='white', borderwidth= 3, relief= 'solid')
    stdBxname.grid(row= 0, column= 1, pady= 2)
    stdLmtr = tkinter.Label(stdFnew, text="Matricula :" , background= "white", font= "Inder 12")
    stdLmtr.grid(row= 1, column= 0, pady= 2, stick='w')
    stdBxmtr = tkinter.Entry(stdFnew, width=20, background='white', borderwidth= 3, relief= 'solid')
    stdBxmtr.grid(row= 1, column= 1, pady= 2)
    stdLid = tkinter.Label(stdFnew, text="senha :" , background= "white", font= "Inder 12")
    stdLid.grid(row= 2, column= 0, pady= 2, stick='w')
    stdBxid = tkinter.Entry(stdFnew, width=20, background='white', borderwidth= 3, relief= 'solid')
    stdBxid.grid(row= 2, column= 1, pady= 2) 
    stdGbutton1 = tkinter.Button(stdFnew, text='Cancelar', command= lambda:[stdBxname.delete(0, tkinter.END),stdBxid.delete(0, tkinter.END),stdBxmtr.delete(0, tkinter.END), closeFrame(stdFnew)] )
    stdGbutton1.grid(row= 3, column= 0, stick='w', padx= 10)

    stdGbutton2 = tkinter.Button(stdFnew, text='Confirmar', command= lambda:newStd(typ) )
    stdGbutton2.grid(row= 3, column= 1, padx= 40)

def delStd(datab):


    global shwSFtlist
    datab= datab.get()
    opendb()
    #curI = shwSFtlist.focus()
    for i in range(len(shwSFtlist.selection())):
        curI = shwSFtlist.selection()[i]
        #curI = ("'"+curI+"'")
        print(curI)
        cur.execute("DELETE FROM %s WHERE id = %s" %(datab, curI))
    con.commit()
    con.close()




#criar as tabelas ALUNO / PROFESSORES / DICIPLINAS
opendb()
cur.execute(""" CREATE TABLE IF NOT EXISTS estudantes(id, nome , matricula , senha ) """)
cur.execute(""" CREATE TABLE IF NOT EXISTS professores(id, nome , matricula , senha ) """)
cur.execute(""" CREATE TABLE IF NOT EXISTS diciplinas( materia, professor, alunos ) """)
con.commit()
con.close()


# criar janela raiz
root = tkinter.Tk()
root.title("Cadastro Institucional ")
root.minsize(width=750, height= 400)

#Frame do estudante
stdFrame = tkinter.Frame(root, height= 400 , width= 750, background='white')
stdSubframe1 = tkinter.Frame(stdFrame, width=25, height= 25, background='Black')
stdSubframe1.pack(fill= 'x', side= 'top', expand= False)
stdSubfLabel = tkinter.Label(stdSubframe1, text='Área dos Alunos', fg='white',bg='black', font=' 14')
stdSubfLabel.pack(side='left')
stdSubframe2 = tkinter.Frame(stdFrame, width=25, height= 25, background='Black')
stdSubframe2.pack(fill= 'x', side= 'top', expand= False)
stdSubf2Button1 = tkinter.Button(stdSubframe2, text='Adicionar', command= lambda:entryFrame(stdFrame,0), bg='black', fg='white', bd=0)
stdSubf2Button1.pack( side= 'left')
stdSubf2Button2 = tkinter.Button(stdSubframe2, text='excluir', command=lambda:[shwList(choosed)], bg='black', fg='white', bd=0)
stdSubf2Button2.pack( side= 'left')
stdSL1 = tkinter.Label(stdSubframe2,padx=100, bg='black')
stdSL1.pack(side='left')
stdSubf2Entry1 = tkinter.Entry(stdSubframe2, width=25, background='black', relief= 'solid', fg='white', highlightbackground='white', highlightthickness= 1)
stdSubf2Entry1.pack(side='left')
stdSubf2Button3 = tkinter.Button(stdSubframe2, text='Pesquisar', command= lambda:[seachStd(stdSubf2Entry1,0)], bg='black', fg='white', bd=0)
stdSubf2Button3.pack( side= 'left')
nowFrame = stdFrame

# Frame do professor
tchFrame = tkinter.Frame(root, height= 400 , width= 750, background='white')
tchSubframe1 = tkinter.Frame(tchFrame, width=25, height= 25, background='Black')
tchSubframe1.pack(fill= 'x', side= 'top', expand= False)
tchSubfLabel = tkinter.Label(tchSubframe1, text='Área dos Professores', fg='white',bg='black', font=' 14')
tchSubfLabel.pack(side='left')
tchSubframe2 = tkinter.Frame(tchFrame, width=25, height= 25, background='Black')
tchSubframe2.pack(fill= 'x', side= 'top', expand= False)
tchSubf2Button1 = tkinter.Button(tchSubframe2, text='Adicionar', command= lambda:entryFrame(tchFrame,1), bg='black', fg='white', bd=0)
tchSubf2Button1.pack( side= 'left')
tchSubf2Button2 = tkinter.Button(tchSubframe2, text='excluir', command=lambda:[shwList(choosed)], bg='black', fg='white', bd=0)
tchSubf2Button2.pack( side= 'left')
tchSL1 = tkinter.Label(tchSubframe2,padx=100, bg='black')
tchSL1.pack(side='left')
tchSubf2Entry1 = tkinter.Entry(tchSubframe2, width=25, background='black', relief= 'solid', fg='white', highlightbackground='white', highlightthickness= 1)
tchSubf2Entry1.pack(side='left')
tchSubf2Button3 = tkinter.Button(tchSubframe2, text='Pesquisar', bg='black', fg='white', command= lambda:[seachStd(stdSubf2Entry1,0,0,0)])
tchSubf2Button3.pack( side= 'left')

#Frame de exibição
shwFrame = tkinter.Frame(root)
sFlabel1 = tkinter.Label(shwFrame, text='Consultar.', font='Arial 12 bold')
sFlabel1.grid(column=0, row=0)
shwSFrame = tkinter.Frame(shwFrame,bg='black')
shwSFrame.grid(row= 2, column=1, stick='n')
sFRSpinbox = tkinter.Spinbox(shwSFrame, values= ('estudantes', 'professores', 'disciplinas'), command= lambda:[shwTyp(),shwList(choosed)],bg='black',fg='white')
sFRSpinbox.grid(row=1, column=1,sticky='nw')
shwSFrame2 = tkinter.Frame(shwFrame)
shwSFrame2.grid(row=3, column=1,pady=10)
shwSFbutton2 = tkinter.Button(shwSFrame, text='Deletar',command=lambda:[delStd(sFRSpinbox),shwList(choosed)], width=10, bg='white')#,state='disabled')
shwSFbutton2.grid(row=2, column=2, sticky='en', padx=10)
shwSFEntry1 = tkinter.Entry(shwSFrame2, width=20, relief='solid' )#highlightthickness=2)
shwSFEntry1.grid(row=0, column=1, sticky='se', padx=10)
shwSFbutton4 = tkinter.Spinbox(shwSFrame2, values=('id', 'nome', 'matricula','senha' ),width=12)
shwSFbutton4.grid(row=0, column=0, sticky='se') 
shwSFbutton3 = tkinter.Button(shwSFrame2, text='Pesquisar: ', command=lambda:seachStd(shwSFEntry1, 1, sFRSpinbox, shwSFbutton4))
shwSFbutton3.grid(row=0 , column= 2, sticky='se')

#resultado da pesquisa



# Barra de menu com funções 
menuOne = tkinter.Menu(stdFrame)
menuOne.add_command(label='H', command= lambda:closeFrame(nowFrame))
menuOne.add_command(label='Alunos', command= lambda:openFrame(stdFrame))
menuOne.add_command(label='Professores', command= lambda: openFrame(tchFrame))
menuOne.add_command(label='Diciplinas', command= newId)#,prtList()])
menuOne.add_command(label='exibir', command=lambda:[shwList(choosed)])
menuOne.add_command(label='fechar', command= lambda:[con.close(), root.destroy()])
root.config(menu = menuOne)




root.mainloop()

linhas a serem validadas : 1 
atual i : 0
row limpa 0
